In [1]:
import numpy as np
#from cupyx.scipy.sparse import csr_matrix as csr_gpu 
import scipy as sc
from tqdm.notebook import tqdm
from scipy import linalg
from scipy.sparse import csr_matrix
from scipy.special import expit as sigmoid
from joblib import Parallel, delayed

In [2]:
import os

from scipy import sparse
import pandas as pd

import bottleneck as bn

import datetime
from copy import deepcopy

In [3]:

import pickle

def save_pkl(obj, filename ):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL )
    
def load_pkl(filename ):
    with open(filename, 'rb') as f:
        return pickle.load(f)
 

In [4]:
DATA_DIR = '/efs/users/hsteck/public/data_for_ease/movielens20mio/'

In [5]:
pro_dir = os.path.join(DATA_DIR, 'pro_sg')


In [6]:
unique_sid = list()
with open(os.path.join(pro_dir, 'unique_sid.txt'), 'r') as f:
    for line in f:
        unique_sid.append(line.strip())

n_items = len(unique_sid)

def load_train_data(csv_file):
    tp = pd.read_csv(csv_file)
    n_users = tp['uid'].max() + 1

    rows, cols = tp['uid'], tp['sid']
    data = sparse.csr_matrix((np.ones_like(rows),
                             (rows, cols)), dtype='float64',
                             shape=(n_users, n_items))
    return data

In [173]:
def load_xtx_binary():
    #items = random.sample(range(20108),1000)
    train_data = load_train_data(os.path.join(pro_dir, 'train.csv'))
    X=train_data[:,:380]
    print (X.shape)
    return [X.shape[0] , X]

In [174]:
userCnt , X=load_xtx_binary()
X_ = X.toarray()
XX_ = 2*X_-1
I = 380
U = 116677

(116677, 380)


In [175]:
def ease(X, lam = 200.0):
    print('multiplying matrix')
    G = X.T @ X
    diagIndices = np.diag_indices(G.shape[0])
    G = G + lam * np.eye(G.shape[0])
    print('inverting')
    P = linalg.inv(G)
    print('inverting complete')
    B = P / (-np.diag(P))
    B[diagIndices] = -1.0
    return B

In [176]:
identity = np.eye(I-1)
identity = csr_matrix(identity)

In [177]:
def irls(item,bias,lam=1.0):
    mask = np.ones(I, dtype=bool)
    mask[item] = False
    w = np.zeros(I-1) 
    for _ in range(2000):
        xw = X[:,mask]@w
        p = sigmoid(xw)
        S = sc.sparse.diags(p*(1-p))
        hess = X[:,mask].T@S@X[:,mask] + lam * identity
        grad = X[:,mask].T@(p-X_[:,item]) + lam*w
        step = np.linalg.solve(hess,grad)
        cov = np.linalg.norm(grad)
        beta = max(1e-32,np.linalg.norm(step))
        wnew = w - sc.special.xlogy(1/beta,1+beta)*step
        #print(cov)
        if cov<1e-11:
            return wnew
        if np.linalg.norm(wnew-w)<1e-12:
            return wnew
        w=wnew
    return wnew



def irls2(item,bias,lam=1.0):
    w = np.zeros(I) 
    mask = np.ones(I, dtype=bool)
    mask[item] = False
    for _ in range(2000):
        xw = X@w
        p = sigmoid(xw)
        S = sc.sparse.diags(p*(1-p))
        hess = X.T@S@X + lam * np.eye(I)
        grad = X.T@(p-X_[:,item]) + lam*w
        #step = sc.sparse.linalg.cg(hess,grad,rtol='1e-10')[0]
        step = np.linalg.solve(hess,grad)
        cov = np.linalg.norm(grad[mask])
        beta = max(1e-32,np.linalg.norm(step))
        wnew = w - sc.special.xlogy(1/beta,1+beta)*step
        #wnew[item] = 0.0
        #print(cov)
        if cov<1e-11:
            return wnew
        if np.linalg.norm(wnew-w)<1e-12:
            return wnew
        w=wnew
    return wnew

In [211]:
#TODO: INCLUDE BIAS
def biased_irls(item,lam=1.0):
    mask = np.ones(I, dtype=bool)
    mask[item] = False
    w = np.zeros(I-1)
    grad = np.zeros(I-1)
    XX = XX_[:,mask]
    #XX = csr_matrix(sparse.hstack((X[:,mask],np.ones(U)[:,None])).A)
    for _ in range(500):
        xw = XX@w
        p = sigmoid(xw)
        S = sc.sparse.diags(p*(1-p))
        hess = XX.T@S@XX + lam * identity
        #w_ = deepcopy(w)
        #w_[-1] = 1e-12*w_[-1]
        grad = XX.T@(p-X_[:,item]) + lam*w
        #step = sc.sparse.linalg.cg(hess,grad,rtol=1e-10)[0]
        step = np.linalg.solve(hess,grad)
        cov = np.linalg.norm(grad)
        beta = np.linalg.norm(step)
        wnew = w - 1/beta*np.log(1+beta) * step
        #new = w - gamma * step
        #print(cov)
        if cov<1e-13:
            return wnew
        if np.linalg.norm(w-wnew)<1e-13:
            return wnew
        w=wnew
    return wnew




In [212]:
def exp_cf(item,bias):
    lam = 200.0
    w = np.zeros(I)
    mask = np.ones(I, dtype=bool)
    mask[item] = False
    w[mask]=biased_irls(item,lam)
    return w

In [188]:
bias = 0.0 
W = Parallel(n_jobs=95)(delayed(exp_cf)(item,bias) for item in tqdm(range(I)))
print('finished')
W = np.array(W)
#np.save('weights_lam200_bias.npy',W.T)
#evaluate(W.T,method='log')

  0%|          | 0/380 [00:00<?, ?it/s]

finished


In [202]:
W_lin = ease(XX,200.0)

multiplying matrix
inverting
inverting complete


In [213]:
evaluate(W_lin,'sq')

2024-07-04 20:15:58.024010
(5000, 380)
0 ... 5000
(5000, 380)
5000 ... 10000
Test NDCG@100=0.27862 (0.00209)
2024-07-04 20:15:58.210922


[0.2786216751573686, nan, nan]

In [214]:
evaluate(W.T,method='sq')

2024-07-04 20:16:01.912233
(5000, 380)
0 ... 5000
(5000, 380)
5000 ... 10000
Test NDCG@100=0.25772 (0.00199)
2024-07-04 20:16:02.101190


[0.2577207250070528, nan, nan]

In [209]:
W

array([[ 0.        , -0.00672722,  0.09687959, ...,  0.06094496,
         0.08619765, -0.01693631],
       [-0.00216831,  0.        ,  0.05196525, ..., -0.05811392,
        -0.06535833, -0.05176961],
       [ 0.07759828,  0.05029494,  0.        , ..., -0.11102305,
        -0.02753693, -0.00614037],
       ...,
       [ 0.06950536, -0.03920781, -0.12356415, ...,  0.        ,
         0.33222109,  0.03567569],
       [ 0.06915554, -0.05715742, -0.0528697 , ...,  0.32514679,
         0.        ,  0.05501154],
       [-0.07194547, -0.04280695, -0.01508004, ...,  0.02153961,
         0.05549289,  0.        ]])

In [15]:
def load_tr_te_data(csv_file_tr, csv_file_te):
    tp_tr = pd.read_csv(csv_file_tr)
    tp_te = pd.read_csv(csv_file_te)

    start_idx = min(tp_tr['uid'].min(), tp_te['uid'].min())
    end_idx = max(tp_tr['uid'].max(), tp_te['uid'].max())

    rows_tr, cols_tr = tp_tr['uid'] - start_idx, tp_tr['sid']
    rows_te, cols_te = tp_te['uid'] - start_idx, tp_te['sid']

    data_tr = sparse.csr_matrix((np.ones_like(rows_tr),
                             (rows_tr, cols_tr)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
    data_te = sparse.csr_matrix((np.ones_like(rows_te),
                             (rows_te, cols_te)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
    return data_tr, data_te

In [16]:
test_data_tr, test_data_te = load_tr_te_data(
    os.path.join(pro_dir, 'test_tr.csv'),
    os.path.join(pro_dir, 'test_te.csv'))

In [17]:
N_test = test_data_tr.shape[0]
idxlist_test = range(N_test)


In [18]:
def NDCG_binary_at_k_batch(X_pred, heldout_batch, k=100):
    '''
    normalized discounted cumulative gain@k for binary relevance
    ASSUMPTIONS: all the 0's in heldout_data indicate 0 relevance
    '''
    batch_users = X_pred.shape[0]
    idx_topk_part = bn.argpartition(-X_pred, k, axis=1)
    topk_part = X_pred[np.arange(batch_users)[:, np.newaxis],
                       idx_topk_part[:, :k]]
    idx_part = np.argsort(-topk_part, axis=1)
    # X_pred[np.arange(batch_users)[:, np.newaxis], idx_topk] is the sorted
    # topk predicted score
    idx_topk = idx_topk_part[np.arange(batch_users)[:, np.newaxis], idx_part]
    # build the discount template
    tp = 1. / np.log2(np.arange(2, k + 2))

    DCG = (heldout_batch[np.arange(batch_users)[:, np.newaxis],
                         idx_topk].toarray() * tp).sum(axis=1)
    IDCG = np.array([(tp[:min(n, k)]).sum()
                     for n in heldout_batch.getnnz(axis=1)])
    return DCG / IDCG

In [19]:
def Recall_at_k_batch(X_pred, heldout_batch, k=100):
    batch_users = X_pred.shape[0]

    idx = bn.argpartition(-X_pred, k, axis=1)
    X_pred_binary = np.zeros_like(X_pred, dtype=bool)
    X_pred_binary[np.arange(batch_users)[:, np.newaxis], idx[:, :k]] = True

    X_true_binary = (heldout_batch > 0).toarray()
    tmp = (np.logical_and(X_true_binary, X_pred_binary).sum(axis=1)).astype(
        np.float32)
    recall = tmp / np.minimum(k, X_true_binary.sum(axis=1))
    return recall

In [196]:
def evaluate(BB,method='sq'):
    #evaluate in batches
    print(datetime.datetime.now())

    #makeSparseFormat(BB, 0.0)


    batch_size_test=5000
    n100_list, r20_list, r50_list = [], [], []



    for bnum, st_idx in enumerate(range(0, N_test, batch_size_test)):
        end_idx = min(st_idx + batch_size_test, N_test)
        Xtest = test_data_tr[idxlist_test[st_idx:end_idx]]
        if method=='log':
            Xtest = 2*Xtest[:,:I].toarray()-1
            print(Xtest.shape)
        else:
            Xtest = Xtest[:,:I].toarray()
        print(Xtest.shape)
        print (str(st_idx)+' ... '+str(end_idx))
        if sparse.isspmatrix(Xtest):
            Xtest = Xtest.toarray()
        Xtest = Xtest.astype('float32')

        #pred_val = Xtest.dot(BB_excl)
        #pred_val = (((Xtest-mu) * scaling).dot(BBth) / scaling) +mu   # no bias
        #pred_val = Xtest.dot(beta_0d)  # no bias
        #pred_val =Xtest.dot(beta_lowrank)  
        if method == 'log':
            pred_val = Xtest.dot(BB)
        else:
            pred_val = Xtest.dot(BB)

        # exclude examples from training and validation (if any)
        pred_val[Xtest[:,:I].nonzero()] = -np.inf
        n100_list.append(NDCG_binary_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=100))
        #r20_list.append(Recall_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=20))
        #r50_list.append(Recall_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=50))
        #calc_coverageCounts(coverageCounts2, pred_val)
        #break  # do only 5000 users

    n100_list = np.concatenate(n100_list)
    #r20_list = np.concatenate(r20_list)
    #r50_list = np.concatenate(r50_list)

    print("Test NDCG@100=%.5f (%.5f)" % (np.mean(n100_list), np.std(n100_list) / np.sqrt(len(n100_list))))
    #print("Test Recall@20=%.5f (%.5f)" % (np.mean(r20_list), np.std(r20_list) / np.sqrt(len(r20_list))))
    #print("Test Recall@50=%.5f (%.5f)" % (np.mean(r50_list), np.std(r50_list) / np.sqrt(len(r50_list))))

    print(datetime.datetime.now())
    return [np.mean(n100_list), np.mean(r20_list), np.mean(r50_list)]

In [182]:
evaluate(W.T,'log')

2024-06-29 20:36:52.145711
(5000, 191)
(5000, 191)
0 ... 5000
(5000, 191)
(5000, 191)
5000 ... 10000
Test NDCG@100=0.19421 (0.00179)
2024-06-29 20:36:52.391011


[0.19421496699521473, nan, nan]

array([[ 0.00000000e+00, -3.39654967e-02,  1.35999950e-01, ...,
        -8.43838352e-05, -8.43838352e-05, -4.63416832e+00],
       [-2.94174137e-02,  0.00000000e+00,  6.23148929e-02, ...,
        -5.43155525e-04, -5.43155525e-04, -3.80532107e+00],
       [ 1.10829447e-01,  5.59055755e-02,  0.00000000e+00, ...,
         2.39191650e-03,  2.39191650e-03, -4.02428046e+00],
       ...,
       [-1.94583648e-04, -4.90338114e-04,  4.60233593e-03, ...,
         4.99973036e-03,  4.99973036e-03, -1.19700308e+01],
       [-1.94583648e-04, -4.90338114e-04,  4.60233593e-03, ...,
         0.00000000e+00,  4.99973036e-03, -1.19700308e+01],
       [-1.94583648e-04, -4.90338114e-04,  4.60233593e-03, ...,
         4.99973036e-03,  0.00000000e+00, -1.19700308e+01]])

In [ ]:
# #Bn_ = (np.ones((I , I)) - 3).flatten()
# Bn = (np.zeros((I , I))).flatten()
# Bn1 = (np.ones((I , I)) + 2).flatten()
# for i in range(10000):
#     #Bnvec = Bn.flatten()
#     #Bn_vec = Bn_.flatten()
#     dBn_ = der(Bn_).flatten()
#     dBn = der(Bn).flatten()
#     top = np.abs(np.inner(Bn - Bn_, dBn - dBn_))
#     bottom = np.linalg.norm(dBn - dBn_) ** 2
#     if bottom <= 1e-132:
#         break
#     gamma = top / bottom
#     #print(gamma)
#     print(np.linalg.norm(der(Bn)))
#     Bn1 = Bn - gamma * der(Bn)
#     Bn_ = Bn
#     Bn = Bn1

# def temp(W):
#     W = W.reshape(I,I)
#     sums = 0.0
#     for i in range(I):
#         xw = X@W[:,i]
#         sums += np.sum( np.logaddexp(-xw/2,xw/2) - X[:,i]/2 * xw)
#     return sums



# def temp2(W, lam = 1.0):
#     W = W.reshape(I,I)
#     S = X @ W
#     return np.sum(np.logaddexp(-S/2,S/2) - X / 2 * S) + lam * np.linalg.norm(W,ord='fro') ** 2


# def der2(W, lam = 1.0):
#     W = W.reshape(I,I)
#     S = X @ W
#     return (1/2 * X.T @ (sigmoid(S/2) - (sigmoid(-S/2)) + X) + 2 * lam * W).flatten()
    



# def der(W):
#     W = W.reshape(I,I)
#     der = np.zeros((I,I))
#     for u in range(U):
#         for i in range(I):
#             scalar = 1 / 2 * (np.tanh(np.inner(X[u],W[:,i])) - X[u,i])
#             der[i] = der[i] + scalar * X[u]
#     return der.flatten()

# def Hess(W):
#     hess = np.zeros((I,I))
#     for u in range(U):
#         for i in range(I):
#             scalar = 1 / (2 * np.cosh(np.inner(X[u],W[:,i])) + 2)
#             hess = hess + scalar * np.outer(X[u],X[u])
#     return hess

# def lsq(B, X, lam = 1.0):
#     B = B.reshape(I,I)
#     S = X @ B
#     loss = sc.linalg.norm(X - S, ord='fro') ** 2 + lam * (sc.linalg.norm(B, ord='fro') ** 2)
#     return loss

# def dlsq(B, X, lam = 1.0):
#     B = B.reshape(I,I)
#     S = X @ B
#     return (2 * lam * B - 2 * X.T @ (X - S)).flatten()


# def llog(Bvec, X, lam = 1.0):
#     B = Bvec.reshape(I,I)
#     S = X @ B
#     Svec = S.flatten()
#     Xvec = X.flatten()

#     loss = np.sum(np.logaddexp(-Svec/2,Svec/2) - Xvec/2*Svec)
    
#     loss = lam * np.inner(Bvec,Bvec) + loss
#     return loss


# def llog2(Bvec, X, lam = 1.0):
#     B = Bvec.reshape(I,I)
#     S = X @ B
#     Svec = S.flatten()
#     Xvec = X.flatten()

#     loss = np.sum(np.logaddexp(-S/2,S/2) - X/2*S)
    
#     loss = lam * np.inner(Bvec,Bvec) + loss
#     return loss

# def dllog2(Bvec, X, lam = 1.0):
#     B = Bvec.reshape(I,I)
#     S = X @ B
#     return (1/2 * X.T @ (2*sigmoid(S) - np.ones((U,I)) + X) + 2 * lam * B).flatten
    
    
     

In [ ]:
# Bn = np.zeros(I * I)  
# Bn_ = np.zeros(I * I) + 10 
# Bn1 = np.zeros(I * I) + 100
# gamma = 1.0
# lam = 0.0
# while gamma > 1e-10:
#     dBn_ = der2(Bn_)
#     dBn = der2(Bn)
#     top = np.abs(np.inner(Bn - Bn_, dBn - dBn_))
#     bottom = np.linalg.norm(dBn - dBn_) ** 2
#     if bottom <= 1e-32:
#         break
#     gamma = top / bottom
#     print(temp2(Bn1),np.linalg.norm(der2(Bn1)),gamma)
#     Bn1 = Bn - gamma * der2(Bn)
#     #Bn1mat = Bn1.reshape(I,I)
#     #Bn1 = (Bn1mat - np.diag(np.diag(Bn1mat))).flatten()
#     Bn_ = Bn
#     Bn = Bn1

In [ ]:
# print(temp2(Bn1),np.linalg.norm(der2(np.eye(I) * 100)))

In [ ]:
# W = ease(X)

In [ ]:
# np.linalg.norm(W - Bn1.reshape(I,I))

In [ ]:
# U = 200_000
# I = 20_000
# lam = 100
# #X = np.zeros((U,I))

# p = np.random.uniform(size=I)
# print('sampling matrix')
# X_ = np.random.binomial(n=1,p=p,size=(U,I)) 
# X = csr_matrix(X_)
# X_ = []

In [ ]:
mask = np.ones(I, dtype=bool)
mask[0] = False
w = np.zeros(I-1)
item = 0

In [ ]:
hess = (np.diag(Var[:,item]**2)@X[:,mask]).T @ X[:,mask]
print('1')
diff=(X[:,mask]@w).reshape(U,1) - X[:,item]
print('2')
grad = X[:,mask].T @ (Var[:,item]**2*diff).T

In [ ]:
w = w-sc.sparse.linalg.spsolve(hess,grad)

In [ ]:
w

In [ ]:
np.multiply(np.multiply(Var[:,item],diff),Var[:,item])

In [ ]:
hess.toar

In [ ]:
X.T.shape

In [105]:
def csr_vappend(a,b):
    """ Takes in 2 csr_matrices and appends the second one to the bottom of the first one. 
    Much faster than scipy.sparse.vstack but assumes the type to be csr and overwrites
    the first matrix instead of copying it. The data, indices, and indptr still get copied."""

    a.data = np.vstack((a.data,b.data))
    a.indices = np.vstack((a.indices,b.indices))
    a.indptr = np.vstack((a.indptr,(b.indptr + a.nnz)[1:]))
    a._shape = (a.shape[0]+b.shape[0],b.shape[1])
    return a

In [106]:
Q=csr_vappend(X,csr_matrix(np.ones(U)))

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 1832034 and the array at index 1 has size 116677

In [130]:
mask = np.ones(I, dtype=bool)
mask[0] = False
Q = csr_matrix(sparse.hstack((X[:,mask],np.ones(U)[:,None])).A)

In [132]:
Q.shape

(116677, 190)